In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import polars as pl
import geopandas as gpd 
import numpy as np
import re
import time

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Ciencia de los datos'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')

In [21]:
file_path = raw / 'DCIS_POPRES1 - Resident population on 1st January - full dataset.csv'
chunk_size = 1000000  # Cantidad de filas por fragmento
parts = 500
i = 0
re_express = re.compile('(\d{1,3})( years)(.*)')
dfs = []
dfs2 = []

In [5]:
# df = pd.read_csv(raw / 'DCIS_POPRES1 - Resident population on 1st January - full dataset.csv', 
#                  sep = ',', decimal = '.', header = 0, encoding = 'utf-8')
# df.head()

In [6]:
# Codigo de dia 1

# # Iterar sobre los fragmentos
# i = 0
# for chunk in pd.read_csv(file_path, chunksize=chunk_size, sep = '|', decimal = '.', header = 0, encoding = 'utf-8'):
#     if i == parts + 1:
#         print('parte: ', i)
#         break
#     else:
#         chunk = chunk[(chunk['TIME'] == 2022) & (chunk['SEXISTAT1'] != 9) 
#                     & (chunk['STATCIV2'] != 99) & (chunk['Age'] != 'total')]
        
#         ter = chunk.copy()
#         ter = ter.loc[:,['ITTER107', 'Territory']]
#         ter['ITTER107'] = ter['ITTER107'].astype(str)
        
#         chunk = chunk.loc[:,['ITTER107', 'SEXISTAT1', 'Age', 'STATCIV2', 'Value']]
#         chunk['Age'] = chunk['Age'].str.replace(re_express, r'\1', regex=True)
#         chunk['ITTER107'] = chunk['ITTER107'].astype(str)
#         chunk['Age'] = chunk['Age'].astype(int)
#         # meter en una lista
#         dfs.append(chunk)
#         dfs2.append(ter)
#         i = i + 1
#         if i%5==0:
#             print(i) 
# # concatenar y guardar
# print('concatenando')
# chunk = pd.concat(dfs, ignore_index=True)
# ter = pd.concat(dfs2, ignore_index=True)
# ter = ter.drop_duplicates()
# print('saving')
# chunk.to_parquet(processed / 'chunk.parquet.gzip', compression='gzip')
# ter.to_parquet(processed / 'ter.parquet.gzip', compression='gzip')

In [22]:
# Iterar sobre los fragmentos
for chunk in pd.read_csv(file_path, chunksize=chunk_size, sep = '|', decimal = '.', header = 0, encoding = 'utf-8'):
    if i == parts + 1:
        print('parte: ', i)
        break
    else:
        chunk = chunk[(chunk['TIME'] == 2022) & (chunk['SEXISTAT1'] != 9) 
                    & (chunk['STATCIV2'] != 99) & (chunk['Age'] != 'total')]
        ter = chunk.copy()
        ter = ter.loc[:,['ITTER107', 'Territory']]
        ter['ITTER107'] = ter['ITTER107'].astype(str)
        chunk = chunk.loc[:,['ITTER107', 'SEXISTAT1', 'Age', 'STATCIV2', 'Value']]
        chunk['Age'] = chunk['Age'].str.replace(re_express, r'\1', regex=True)
        chunk['ITTER107'] = chunk['ITTER107'].astype(str)
        chunk['Age'] = chunk['Age'].astype(int)
        # meter en una lista
        dfs.append(chunk)
        dfs2.append(ter)
        i = i + 1
        if i%5==0:
            print(i) 
            # concatenar y guardar
            print('concatenando')
            chunk = pd.concat(dfs, ignore_index=True)
            ter = pd.concat(dfs2, ignore_index=True)
            ter = ter.drop_duplicates()
            print('saving')
            chunk.to_parquet(processed / 'chunk.parquet.gzip', compression='gzip')
            ter.to_parquet(processed / 'ter.parquet.gzip', compression='gzip')
           
# # concatenar y guardar
# print('concatenando')
# chunk = pd.concat(dfs, ignore_index=True)
# ter = pd.concat(dfs2, ignore_index=True)
# ter = ter.drop_duplicates()
# print('saving')
# chunk.to_parquet(processed / 'chunk.parquet.gzip', compression='gzip')
# ter.to_parquet(processed / 'ter.parquet.gzip', compression='gzip')

KeyboardInterrupt: 

In [8]:
italy = pd.read_parquet(processed / 'chunk.parquet.gzip')
italy.head()

,ITTER107,SEXISTAT1,Age,STATCIV2,Value
0,IT,1,0,1,205371.0
1,IT,2,0,1,195255.0
2,ITCD,1,0,1,94721.0
3,ITCD,2,0,1,89834.0
4,ITC,1,0,1,53931.0


In [9]:
italy['Age'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [10]:
italy.shape

(6603221, 5)

In [11]:
italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6603221 entries, 0 to 6603220
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   ITTER107   object 
 1   SEXISTAT1  int64  
 2   Age        int64  
 3   STATCIV2   int64  
 4   Value      float64
dtypes: float64(1), int64(3), object(1)
memory usage: 251.9+ MB


In [12]:
ter = pd.read_parquet(processed / 'ter.parquet.gzip')
ter.head(10)

,ITTER107,Territory
0,IT,Italy
2,ITCD,Nord
4,ITC,Nord-ovest
6,ITC1,Piemonte
8,ITC11,Torino
10,ITC12,Vercelli
12,ITC13,Biella
14,ITC14,Verbano-Cusio-Ossola
16,ITC15,Novara
18,ITC16,Cuneo


In [13]:
ter[ter['Territory'] == 'Cuneo']

,ITTER107,Territory
18,ITC16,Cuneo
278,004078,Cuneo
1445399,4078,Cuneo


In [14]:
ter.shape

(15203, 2)

In [15]:
# kk = italy[italy['Territory'] == 'Aprica']
# # kk = kk.sort_values(by=['Age','Gender'], ascending=True)
# kk.head(10)

In [16]:
# start = time.time()
# df = pl.read_csv(file_path).lazy().filter(
#          (pl.col('TIME') == 2022) & 
#          (pl.col('SEXISTAT1') != 9) &
#          (pl.col('STATCIV2') != 99)).collect()
# end = time.time()
# print(end - start)
# display(df)

In [17]:
# geo = gpd.read_file(raw / "Limiti01012023_g/Com01012023_g/Com01012023_g_WGS84.shp")
# geo.geometry = geo.geometry.to_crs(epsg = 4326)
# # geo = geo.loc[:,['geometry','COD_CM','COMUNE']]
# geo.head()

In [18]:
# geo[geo['PRO_COM_T'] == '014004']

SEXISTAT1

1- Male   <br>
2- Female <br>
9- Total  <br>

STATCIV2
1- never married <br>
2- married       <br>
3- divorced      <br>
4- widowed       <br>
99- total        <br>

columns of italy <br>
Index(['ITTER107', 'Territory', 'TIPO_DATO15', 'Demographic data type',
       'SEXISTAT1', 'Gender', 'ETA1', 'Age', 'STATCIV2', 'Marital status',
       'TIME', 'Select time', 'Value', 'Flag Codes', 'Flags'],
      dtype='object')

In [19]:
print('Ok_')

Ok_
